# Реализация LU- и LUP- алгоритмов для решения СЛАУ
## Работу выполнила Балакирева Дарья НПМбд-01-19

Перед написанием кода импортируем библиотеку numpy.

In [2]:
import numpy as np

## LU-алгоритм

Реализуем алгоритм превращения данной на вход матрицы в L и U матрицы. Сначала зададим матрицу U как копию A, а матрицу L как единичную матрицу размером n x n, а затем заполним их, используя вспомогательную переменную r.

In [3]:
def From_A_to_LU(A):
    s = A.shape[0]
    U = A.copy()
    L = np.eye(s)
    
    for i in range(s):
        r = U[i+1:, i] / U[i, i]
        L[i+1:, i] = r
        U[i+1:] -= r[:, np.newaxis] * U[i]
        
    return L, U

Получив L и U, мы можем найти наше решение в два этапа - сначала мы находим y из L*y = b:

In [4]:
def get_y(L, b):   
    s = L.shape[0]    
    y = np.zeros_like(b, dtype=np.double);   
    y[0] = b[0] / L[0, 0]    
    for i in range(1, s):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]        
    return y

А затем из уравнения U*x = y мы находим x:

In [5]:
def get_x(U, y):
    s = U.shape[0]
    x = np.zeros_like(y, dtype=np.double);
    x[-1] = y[-1] / U[-1, -1]
    for i in range(s-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]   
    return x

Финальная функция решения будет выглядеть так:

In [6]:
def LU_solve(A, b):  
    L, U = From_A_to_LU(A) 
    y = get_y(L, b)
    return get_x(U, y)

Протестируем наш код:

In [7]:
A = np.array([[1., 2., 3.], [4., 5., 6.], [7., 8., 10.]])
b = np.array([[1], [0], [0]])

print(LU_solve(A, b))
%timeit LU_solve(A,b)

[[-0.66666667]
 [-0.66666667]
 [ 1.        ]]
102 µs ± 5.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Сверимся со встроенной функцией, реализованной в numpy:

In [8]:
np.linalg.solve(A, b)

array([[-0.66666667],
       [-0.66666667],
       [ 1.        ]])

## PLU-алгоритм
Отличия будут только в функции превращения матрицы A в теперь уже 3 новые матрицы P, L, U, а также в solve-функции. Здесь в цикле for происходит исключительно перестановка элементов внутреннего цикла k, поэтому цикл будет не таким медленным.

In [9]:
def From_A_to_PLU(A):
    s = A.shape[0]
    U = A.copy()
    L = np.eye(s, dtype=np.double)
    P = np.eye(s, dtype=np.double)
    for i in range(s):
        for j in range(i, s):
            #для избежания погрешностей при очень близком к нулю значении
            if ~np.isclose(U[i, i], 0.0):
                break
            U[[j, j+1]] = U[[j+1, j]]
            P[[j, j+1]] = P[[j+1, j]]
        r = U[i+1:, i] / U[i, i]
        L[i+1:, i] = r
        U[i+1:] -= r[:, np.newaxis] * U[i]   
    return P, L, U

Реализация самой solve-функции не сильно отличается:

In [10]:
def PLU_solve(A, b):
    P, L, U = From_A_to_PLU(A)
    y = get_y(L, np.dot(P, b))
    return get_x(U, y)

Тестируем наш код:

In [25]:
A = np.array([[1., 2., 3.], [4., 5., 6.], [7., 8., 10.]])
b = np.array([[1], [0], [0]])

print(PLU_solve(A, b))
%timeit PLU_solve(A,b)

[[-0.66666667]
 [-0.66666667]
 [ 1.        ]]
304 µs ± 36.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Сверяем со встроенной функцией:

In [22]:
np.linalg.solve(A, b)

array([[-2.66666667],
       [ 3.33333333],
       [-1.        ]])

# Вывод:
В разобранном нами примере функция LU работает быстрее PLU приблизительно в 3 раза, но обе функции делают это достаточно быстро. Мы разобрались в реализации алгоритмов на языке python.